In [2]:
# %load neo_quant.py
import pandas as pd
import numpy as np
import requests
import bs4
import time
import datetime
import matplotlib.pyplot as plt
from dateutil import parser
from matplotlib import font_manager, rc
from IPython.display import HTML
pd.set_option('display.max_colwidth', -1)
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
pd.options.display.max_rows = 60
%matplotlib inline

my_portfolio = ['동국제강', '포스코','현대건설','GS건설','한국조선해양','OCI','미래에셋대우','삼성증권','삼성중공업','키움증권','LG화학']
my_portfolio_code_list = ['A001230','A005490','A000720','A006360','A009540','A010060','A006800','A016360','A010140','A039490','A051910']

def make_code(x):
    x = str(x)
    return 'A' + '0' * (6-len(x)) + x


def make_code2(x):
    x = str(x)
    return '0' * (6-len(x)) + x


def get_all_company_info():
    kospi = pd.read_excel('data/kospi.xls')
    kosdaq = pd.read_excel('data/kosdaq.xls')
    kospi['구분'] = '코스피'
    kosdaq['구분'] = '코스닥'
    companies = pd.concat([kospi, kosdaq])
    companies = companies[['구분','종목코드','기업명']]
#     companies['종목코드'] = companies['종목코드'].apply(modifyCompanyCode)
    companies = companies.reset_index()
    return companies

def get_company_data(min_price=0):
    kospi = pd.read_excel('data/kospi.xls')
    kosdaq = pd.read_excel('data/kosdaq.xls')
    kospi['구분'] = '코스피'
    kosdaq['구분'] = '코스닥'
    companies = pd.concat([kospi, kosdaq])
    companies = companies[['종목코드','기업명','구분','업종코드','업종','액면가(원)','상장주식수(주)', '자본금(원)']]
    companies['상장주식수(주)'] = companies['상장주식수(주)'].str.replace(',','').astype(int)
    companies['자본금(원)'] = companies['자본금(원)'].str.replace(',','').astype(int)
    companies['액면가(원)'] = companies['액면가(원)'].str.replace(',','').astype(float)
    companies = companies[companies['액면가(원)'] >= min_price]
    companies['액면가(원)'] = companies['액면가(원)'].astype(int)
    companies = companies.set_index('종목코드')
    companies = company_apply_a_prefix(companies)
    return companies

def company_apply_a_prefix(companies):
    old_index = pd.Series(companies.index)
    new_index = old_index.map(make_code)
    return companies.set_index(new_index)

def company_apply_no_prefix(companies):
    old_index = pd.Series(companies.index)
    new_index = old_index.map(make_code2)
    return companies.set_index(new_index)

def remove_a_from_company_code(company_df):
    code_list = []
    for code in company_df.index:
        code = code.replace('A','')
        code_list.append(code)
    return company_df.set_index(pd.Series(code_list))

def get_price_data():
    price_path = r'data/price_data_2013.xlsx'
    price_df = pd.read_excel(price_path)
    price_df = price_df.set_index(price_df.columns[0])
#     price_df.columns = 'A' + price_df.columns 
    return price_df

def get_fs_data():
    fs_path = r'data/fs_data.xlsx'
    return get_finance_data(fs_path)

def get_fr_data():
    fr_path = r'data/fr_data.xlsx'
    return get_finance_data(fr_path)

def get_invest_data():
    invest_path = r'data/invest_data.xlsx'
    return get_finance_data(invest_path)

def load_all_data():
    companies = get_company_data()
    prices = get_price_data()
    fs_df = get_fs_data()
    fr_df = get_fr_data()
    iv_df = get_invest_data()
    return companies, prices, fs_df, fr_df, iv_df
    
# 액면가 1000원 이상 회사 리스트 가져와서 코드앞에 A 붙이기
# companies = get_company_info(1000)
# companies = apply_a_type_code(companies)

def filter_company(st_df, companies):
    new_company_list = []
    for num, code in enumerate(st_df.index):
        if code in companies.index:
            new_company_list.append(code)

    return st_df.loc[new_company_list]

def add_company_info(st_df, companies):
    return pd.merge(st_df, companies, how='inner', left_index=True, right_index=True)

# 데이터프레임에서 회사코드로 필터링한 후 회사 정보 추가하기
# roa_filter_info = add_company_info(filter_company(roa, companies), companies)

def get_kospi_list(st_df):
    return st_df[st_df['구분']=='코스피']

def get_kosdaq_list(st_df):
    return st_df[st_df['구분']=='코스닥']

def get_price_over_list(st_df, price):
    return st_df[st_df['액면가(원)'] >= price]

def get_company_code(name, company_df):
    return company_df[company_df['기업명']==name].index[0]

def get_company_code_list(company_name_list, company_df):
    code_list = []
    for company_name in company_name_list:
        for num, name in enumerate(company_df['기업명']):
            if company_name in name:
                code_list.append({'code':company_df.index[num], 'name':name})
    return code_list

def get_company_name(company_code, company_df):
    return company_df.loc[company_code]['기업명']

def show_chart(company_name_list, company_df, price_df, year_duration=1):
    end_date = price_df.iloc[-1].name
    start_date = end_date - datetime.timedelta(days=year_duration * 365)
    company_list = get_company_code_list(company_name_list, company_df)
    if len(company_list) == 0:
        print('no company with name' + company_name)
        return
    code = company_list[0]['code']
    name = company_list[0]['name']
#     code = code.replace('A','')
    strategy_price = price_df[code][start_date:end_date]
    strategy_df = pd.DataFrame({'price':strategy_price})
    plt.figure(figsize=(10, 6))
    strategy_df['price'].plot(label=name)
    plt.legend()
    plt.show()          

def show_multi_chart(company_code_list, price_df, company_df, year_duration=1):
    end_date = price_df.iloc[-1].name
    start_date = end_date - datetime.timedelta(days=year_duration * 365)
    strategy_price = price_df[company_code_list][start_date:end_date]
    num_row = int((len(company_code_list)-1)/2)+1
    plt.figure(figsize=(10, num_row*5))
    for i, code in enumerate(company_code_list):
        ax = plt.subplot(num_row, 4, i+1)
        name = get_company_name(code, company_df)
        ax.title.set_text(name)
        ax.plot(strategy_price.index, strategy_price[code])
    plt.show()
    
def show_detail_chart(company_name, company_df, price_df, year_duration=1):
    end_date = price_df.iloc[-1].name
    start_date = end_date - datetime.timedelta(days=year_duration * 365)
    company_list = get_company_code_list(company_name, company_df)
    if len(company_list) == 0:
        print('no company with name' + company_name)
        return
    code = company_list[0]['code']
    name = company_list[0]['name']
#     code = code.replace('A','')
    strategy_price = price_df[code][start_date:end_date]
    strategy_df = pd.DataFrame({'price':strategy_price})
    strategy_df
    ma5 = strategy_df['price'].rolling(window=5).mean()
    strategy_df['ma5'] = ma5
    ma10 = strategy_df['price'].rolling(window=10).mean()
    strategy_df['ma10'] = ma10
    ma20 = strategy_df['price'].rolling(window=20).mean()
    strategy_df['ma20'] = ma20
    ma60 = strategy_df['price'].rolling(window=60).mean()
    strategy_df['ma60'] = ma60
    ma120 = strategy_df['price'].rolling(window=120).mean()
    strategy_df['ma120'] = ma120
    plt.figure(figsize=(20, 12))
    # strategy_df['price'].plot(label=name)
    plt.plot(strategy_df.index, strategy_df['price'], color='darkblue',linewidth=3.0)
    plt.plot(strategy_df.index, strategy_df['ma5'], color='red', label='ma5')
    plt.plot(strategy_df.index, strategy_df['ma10'], color='blue', label='ma10')
    plt.plot(strategy_df.index, strategy_df['ma20'], color='green', label='ma20')
    plt.plot(strategy_df.index, strategy_df['ma60'], color='cyan', label='ma60')
    plt.plot(strategy_df.index, strategy_df['ma120'], color='yellow', label='ma120')
    plt.title(name)
    plt.xlabel("duration")
    plt.ylabel("price")
    plt.legend(loc='upper right')
    plt.grid()
    plt.show() 
    
    
def get_maximum_earning_rate(price_df, company_df, year_duration=1, min_price=0, type='all'):
    end_date = price_df.iloc[-1].name
    start_date = end_date - datetime.timedelta(days=year_duration * 365)
    strategy_price = price_df[start_date:end_date]
    strategy_price = strategy_price.fillna(method='bfill')
    last_price = strategy_price.iloc[-1]
    first_price = strategy_price.iloc[0]
    price_diff_df = pd.DataFrame({first_price.name:first_price, last_price.name:last_price})
#     price_diff_df.index = 'A' + price_diff_df.index
    price_diff_df['diff'] = price_diff_df[last_price.name] - price_diff_df[first_price.name]
    price_diff_df = price_diff_df[price_diff_df[last_price.name] > 5000]
    price_diff_df = price_diff_df[price_diff_df['diff'] > 0]
    # price_diff_df['ratio'] = price_diff_df['diff'] / price_diff_df[first_price.name]
    price_diff_df['ratio'] = ((price_diff_df[last_price.name] / price_diff_df[first_price.name]) - 1) * 100
    price_diff_df['ratio'] = price_diff_df['ratio'].astype(int)
    price_diff_df = price_diff_df.sort_values(by='ratio', ascending=False)
    price_diff_df = add_company_info(price_diff_df, company_df)
    if type == 'kospi':
        price_diff_df = get_kospi_list(price_diff_df)
    elif type == 'kosdaq':
        price_diff_df = get_kosdaq_list(price_diff_df)
    return price_diff_df
#     price_diff_df['fs_info'] = price_diff_df.index
#     price_diff_df['fs_info'] = price_diff_df['fs_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode={0}" target="_blank">fs</a>'.format(x))
#     price_diff_df['fr_info'] = price_diff_df.index
#     price_diff_df['fr_info'] = price_diff_df['fr_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode={0}" target="_blank">fr</a>'.format(x))
#     price_diff_df['iv_info'] = price_diff_df.index
#     price_diff_df['iv_info'] = price_diff_df['iv_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode={0}" target="_blank">iv</a>'.format(x))
#     return HTML(price_diff_df.to_html(escape=False))

def show_pf_earning_rate(code_list, price_df, year_duration=1, initial_monehy=100000000):
    end_date = price_df.iloc[-1].name
    start_date = end_date - datetime.timedelta(days=year_duration * 365)
    st_backtest = backtest_with_code_list(price_df, code_list, start_date, end_date, initial_money)
    plt.figure(figsize=(10, 6))
    st_backtest['총변화율'].plot()
    plt.show()
    
def show_company_info(company_code_list, company_df):
    firm_df = company_df.loc[company_code_list]
    firm_df['fs_info'] = firm_df.index
    firm_df['fs_info'] = firm_df['fs_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode={0}" target="_blank">fs</a>'.format(x))
    firm_df['fr_info'] = firm_df.index
    firm_df['fr_info'] = firm_df['fr_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode={0}" target="_blank">fr</a>'.format(x))
    firm_df['iv_info'] = firm_df.index
    firm_df['iv_info'] = firm_df['iv_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode={0}" target="_blank">iv</a>'.format(x))
    return HTML(firm_df.to_html(escape=False))
    
def show_company_info_from_name(firm_name, company_df):
    company_list = get_company_code_list(firm_name, company_df)
    if len(company_list) == 0:
        print('no company with name' + company_name)
        return
    company_list
    code_list = []
    for company in company_list:
        code_list.append(company['code'])
    return show_company_info(code_list, companies)

def get_earning_rate(firm_name, company_df, price_df, year_duration=1):
    code_list = get_company_code_list(firm_name, company_df)
    if len(code_list) == 0:
        return "No Company with name : " + firm_name
    name = code_list[0]['name']
    code = code_list[0]['code']
#     code = code.replace('A','')
    end_date = price_df.iloc[-1].name
    start_date = end_date - datetime.timedelta(days=year_duration * 365)
    strategy_price = price_df[code][start_date:end_date]
    strategy_price = strategy_price.dropna()
    last_price = strategy_price.iloc[-1]
    first_price = strategy_price.iloc[0]
    print(str(strategy_price.index[0])+" : "+str(first_price))
    print(str(strategy_price.index[-1])+" : "+str(last_price))
    profit = int((last_price/first_price - 1) * 100)
    return name + " : " + str(profit) + '%'

def get_vaild_code_from_price_df(code_list, price_df):
    new_code_list = []
    for code in code_list:
        if code in price_df.columns:
            new_code_list.append(code)
    return new_code_list        

def show_business_trend(company_df, price_df, year_duration=1):
    end_date = price_df.iloc[-1].name
    start_date = end_date - datetime.timedelta(days=year_duration * 365)

    company_df['업종코드'] = company_df['업종코드'].astype(str)
    company_df['업종구분'] = company_df['업종코드'] + ' (' + company_df['업종'] + ')'
    all_busi = company_df['업종구분'].unique()
    busi_list = []
    for busi_name in all_busi:
        temp_list = company_df[company_df['업종구분'] == busi_name]
        busi_list.append({'name':busi_name + ' - ' + str(len(temp_list)),'list':temp_list.index})

    for i, busi in enumerate(busi_list):
        busi_name = busi['name']
        busi_code_list = busi['list']
        strategy_price = price_df[get_vaild_code_from_price_df(busi_code_list, price_df)][start_date:end_date]
        strategy_price.fillna(method='bfill')
        busi_earning = strategy_price.sum(axis=1)
        temp_df = pd.DataFrame({busi_name:busi_earning})
        if i == 0:
            total_busi_earning = temp_df
        else:
            total_busi_earning = pd.merge(total_busi_earning, temp_df, how='outer', left_index=True, right_index=True)
    num_row = int((len(total_busi_earning.columns)-1)/4)+1
    plt.figure(figsize=(6*4, num_row*6))
    for i, busi in enumerate(total_busi_earning.columns):
        ax = plt.subplot(num_row, 4, i+1)
        ax.title.set_text(busi)
        ax.plot(total_busi_earning.index, total_busi_earning[busi])
    plt.show()    
    
def get_company_list_from_business_code(busi_code, company_df):
    return company_df[company_df['업종코드'] == busi_code].index

# 기존 price dataframe 을 최신 가격으로 업데이트.
# ex) prices_update = update_prices(companies, prices, '10') 
# 기존 price 와 신규 price 의 차이 비교
# prices_update.merge(prices,indicator = True, left_index=True, right_index=True, how='outer').loc[lambda x : x['_merge']!='both']
def update_origin_price_df(origin_price_df, code, count):
    origin_prices = origin_price_df
    price_df = make_price_dataframe(code, 'day', count)
    price_df.index = pd.to_datetime(price_df.index)
    price_df[code] = price_df[code].astype(float)
    price_df_origin = pd.DataFrame({code:origin_prices[code]}) 
    new_price_df = pd.concat([price_df_origin, price_df])
    new_price_df = new_price_df.loc[~new_price_df.index.duplicated(keep='last')]
    origin_prices = origin_prices.drop(code, axis=1)
    origin_prices = pd.merge(origin_prices, new_price_df, left_index=True, right_index=True, how='outer')
    return origin_prices
    
def update_prices(company_df, origin_price_df, count):
    origin_prices = origin_price_df
    for num, code in enumerate(company_df.index):
        try:
            print(num, code)
            time.sleep(1)
            try:
                origin_prices = update_origin_price_df(origin_prices, code, count)
            except requests.exceptions.Timeout:
                time.sleep(60)
                origin_prices = update_origin_price_df(origin_prices, code, count)
        except ValueError:
            continue
        except KeyError:
            continue
    return origin_prices

# [코드 3.15] 재무제표 데이터를 가져와 데이터프레임으로 만드는 함수 (CH3. 데이터 수집하기.ipynb)

def make_fs_dataframe(firm_code):
    fs_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode=' + firm_code
    fs_page = requests.get(fs_url)
    fs_tables = pd.read_html(fs_page.text)
    
    temp_df = fs_tables[0]
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df[temp_df.columns[:4]]
    temp_df = temp_df.loc[['매출액', '영업이익', '당기순이익']]

    temp_df2 = fs_tables[2]
    temp_df2 = temp_df2.set_index(temp_df2.columns[0])
    temp_df2 = temp_df2.loc[['자산', '부채', '자본']]

    temp_df3 = fs_tables[4]
    temp_df3 = temp_df3.set_index(temp_df3.columns[0])
    temp_df3 = temp_df3.loc[['영업활동으로인한현금흐름']]

    fs_df = pd.concat([temp_df, temp_df2, temp_df3])
    
    return fs_df

# [코드 3.19] 데이터프레임 형태 바꾸기 코드 함수화 (CH3. 데이터 수집하기.ipynb)

def change_df(firm_code, dataframe):
    for num, col in enumerate(dataframe.columns):
        temp_df = pd.DataFrame({firm_code : dataframe[col]})
        temp_df = temp_df.T
        temp_df.columns = [[col]*len(dataframe), temp_df.columns]
        if num == 0:
            total_df = temp_df
        else:
            total_df = pd.merge(total_df, temp_df, how='outer', left_index=True, right_index=True)    
    
    return total_df

# [코드 3.21] 재무 비율 데이터프레임을 만드는 함수 (CH3. 데이터 수집하기.ipynb)

def make_fr_dataframe(firm_code):
    fr_url = 'https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode=' + firm_code
    fr_page = requests.get(fr_url)
    fr_tables = pd.read_html(fr_page.text)
    
    temp_df = fr_tables[0]
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df.loc[['유동비율계산에 참여한 계정 펼치기',
                           '부채비율계산에 참여한 계정 펼치기',
                           '영업이익증가율계산에 참여한 계정 펼치기',
                           'ROA계산에 참여한 계정 펼치기',
                           'ROIC계산에 참여한 계정 펼치기']]
    temp_df.index = ['유동비율', '부채비율', '영업이익률', 'ROA', 'ROIC']
    return temp_df

# [코드 3.23] 투자지표 데이터프레임을 만드는 함수 (CH3. 데이터 수집하기.ipynb)

def make_invest_dataframe(firm_code):
    invest_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode=' + firm_code
    invest_page = requests.get(invest_url)
    invest_tables = pd.read_html(invest_page.text)
    temp_df = invest_tables[1]
    
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df.loc[['PER계산에 참여한 계정 펼치기',
                       'PCR계산에 참여한 계정 펼치기',
                       'PSR계산에 참여한 계정 펼치기',
                       'PBR계산에 참여한 계정 펼치기',
                      '총현금흐름']]
    temp_df.index = ['PER', 'PCR', 'PSR', 'PBR', '총현금흐름']
    return temp_df

#  [코드 3.40] 가격을 가져와 데이터프레임 만드는 함수 (CH3. 데이터 수집하기 2.ipynb)

def make_price_dataframe(code, timeframe, count):
    if code.startswith('A'):
        request_code = code.replace('A','')
    url = 'https://fchart.stock.naver.com/sise.nhn?requestType=0'
    price_url = url + '&symbol=' + request_code + '&timeframe=' + timeframe + '&count=' + count
    price_data = requests.get(price_url)
    price_data_bs = bs4.BeautifulSoup(price_data.text, 'lxml')
    item_list = price_data_bs.find_all('item')
    
    date_list = []
    price_list = []
    for item in item_list:
        temp_data = item['data']
        datas = temp_data.split('|')
        date_list.append(datas[0])
        price_list.append(datas[4])

    price_df = pd.DataFrame({code:price_list}, index=date_list)
    
    return price_df

#  [코드 4.6] 재무 데이터 전처리하는 함수 (CH4. 전략 구현하기.ipynb)

def get_finance_data(path):
    data_path = path
    raw_data = pd.read_excel(data_path)
    raw_data = raw_data.set_index(raw_data.columns[0])
    big_col = list(raw_data.columns)
    small_col = list(raw_data.iloc[0])
    
    new_big_col = []
    for num, col in enumerate(big_col):
        if 'Unnamed' in col:
            new_big_col.append(new_big_col[num-1])
        else:
            new_big_col.append(big_col[num])
            
    raw_data.columns = [new_big_col, small_col]
    clean_df = raw_data.loc[ raw_data.index.dropna() ]
    
    return clean_df

def check_IFRS(x):
    if x == 'N/A(IFRS)':
        return np.NaN
    else:
        return x

#  [코드 4.14] PER기준으로 오름차순으로 정렬하여 주는 함수 (CH4. 전략 구현하기.ipynb)

def low_per(invest_df, index_date, num):
    invest_df[(index_date, 'PER')] = pd.to_numeric(invest_df[(index_date, 'PER')])
    per_sorted = invest_df.sort_values(by=(index_date, 'PER'))
    return per_sorted[index_date][:num]

#  [코드 4.15] ROA기준으로 내림차순으로 정렬하여 주는 함수 (CH4. 전략 구현하기.ipynb)

def high_roa(fr_df, index_date, num):
    fr_df[(index_date, 'ROA')] = fr_df[(index_date, 'ROA')].apply(check_IFRS)
    fr_df[(index_date, 'ROA')] = pd.to_numeric(fr_df[(index_date, 'ROA')] )
    sorted_roa = fr_df.sort_values(by=(index_date, 'ROA'), ascending=False)
    return sorted_roa[index_date][:num]


#  [코드 4.22] 마법공식 함수로 만들기 (CH4. 전략 구현하기.ipynb)

def magic_formula(fr_df, invest_df, index_date, num):
    per = low_per(invest_df, index_date, None)
    roa = high_roa(fr_df, index_date, None)
    per['per순위'] = per['PER'].rank()
    roa['roa순위'] = roa['ROA'].rank(ascending=False)
    magic = pd.merge(per, roa, how='outer', left_index=True, right_index=True)
    magic['마법공식 순위'] = (magic['per순위'] + magic['roa순위']).rank().sort_values()
    magic = magic.sort_values(by='마법공식 순위')
    return magic[:num]

#  [코드 4.23] 저평가 지수를 기준으로 정렬하여 순위 만들어 주는 함수 (CH4. 전략 구현하기.ipynb)

def get_value_rank(invest_df, value_type, index_date, num):
    invest_df[(index_date,  value_type)] = pd.to_numeric(invest_df[(index_date,  value_type)])
    value_sorted = invest_df.sort_values(by=(index_date,  value_type))[index_date]
    value_sorted[  value_type + '순위'] = value_sorted[value_type].rank()
    return value_sorted[[value_type, value_type + '순위']][:num]

#  [코드 4.25] 저평가 지표 조합 함수 (CH4. 전략 구현하기.ipynb)

def make_value_combo(value_list, invest_df, index_date, num):
    
    for i, value in enumerate(value_list):
        temp_df = get_value_rank(invest_df, value, index_date, None)
        if i == 0:
            value_combo_df = temp_df
            rank_combo = temp_df[value + '순위']
        else:
            value_combo_df = pd.merge(value_combo_df, temp_df, how='outer', left_index=True, right_index=True)
            rank_combo = rank_combo + temp_df[value + '순위']
    
    value_combo_df['종합순위'] = rank_combo.rank()
    value_combo_df = value_combo_df.sort_values(by='종합순위')
    
    return value_combo_df[:num]

#  [코드 4.29] F-score 함수(CH4. 전략 구현하기.ipynb)

def get_fscore(fs_df, index_date, num):
    fscore_df = fs_df[index_date]
    fscore_df['당기순이익점수'] = fscore_df['당기순이익'] > 0
    fscore_df['영업활동점수'] = fscore_df['영업활동으로인한현금흐름'] > 0
    fscore_df['더큰영업활동점수'] = fscore_df['영업활동으로인한현금흐름'] > fscore_df['당기순이익']
    fscore_df['종합점수'] = fscore_df[['당기순이익점수', '영업활동점수', '더큰영업활동점수']].sum(axis=1)
    fscore_df = fscore_df[fscore_df['종합점수'] == 3]
    return fscore_df[:num]

#  [코드 4.34] 모멘텀 데이터프레임 만들기 함수화 (CH4. 전략 구현하기.ipynb)

def get_momentum_rank(price_df, index_date, date_range, num):
    momentum_df = pd.DataFrame(price_df.pct_change(date_range).loc[index_date])
    momentum_df.columns = ['모멘텀']
    momentum_df['모멘텀순위'] = momentum_df['모멘텀'].rank(ascending=False)
    momentum_df = momentum_df.sort_values(by='모멘텀순위')
    return momentum_df[:num]

#  [코드 4.39] 저평가 + Fscore 함수화 (CH4. 전략 구현하기.ipynb)

def get_value_quality(invest_df, fs_df, index_date, num):
    value = make_value_combo(['PER', 'PBR', 'PSR', 'PCR'], invest_df, index_date, None)
    quality = get_fscore(fs_df, index_date, None)
    value_quality = pd.merge(value, quality, how='outer', left_index=True, right_index=True)
    value_quality_filtered = value_quality[value_quality['종합점수'] == 3]
    vq_df = value_quality_filtered.sort_values(by='종합순위')
    return vq_df[:num]

#  [코드 5.12] 백테스트 함수 버젼1 (Ch5. 백테스트.ipynb)


def backtest_beta(price_df, strategy_df, start_date, end_date, initial_money):

    code_list = []
    for code in strategy_df.index:
#         code_list.append(code.replace('A',''))
        code_list.append(code)

    strategy_price = price_df[code_list][start_date:end_date]

    pf_stock_num = {}
    stock_amount = 0
    stock_pf = 0
    each_money = initial_money / len(strategy_df)
    for code in strategy_price.columns:
        temp = int( each_money / strategy_price[code][0] )
        pf_stock_num[code] = temp
        stock_amount = stock_amount + temp * strategy_price[code][0]
        stock_pf = stock_pf + strategy_price[code] * pf_stock_num[code]

    cash_amount = initial_money - stock_amount

    backtest_df = pd.DataFrame({'주식포트폴리오':stock_pf})
    backtest_df['현금포트폴리오'] = [cash_amount] * len(backtest_df)
    backtest_df['종합포트폴리오'] = backtest_df['주식포트폴리오'] + backtest_df['현금포트폴리오']
    backtest_df['일변화율'] = backtest_df['종합포트폴리오'].pct_change()
    backtest_df['총변화율'] = backtest_df['종합포트폴리오']/initial_money - 1
    
    return backtest_df

def backtest_with_code_list(price_df, code_list_to_test, start_date, end_date, initial_money):

    code_list = []
    for code in code_list_to_test:
#         code_list.append(code.replace('A',''))
        code_list.append(code)

    strategy_price = price_df[code_list][start_date:end_date]

    pf_stock_num = {}
    stock_amount = 0
    stock_pf = 0
    each_money = initial_money / len(code_list)
    for code in strategy_price.columns:
        temp = int( each_money / strategy_price[code][0] )
        pf_stock_num[code] = temp
        stock_amount = stock_amount + temp * strategy_price[code][0]
        stock_pf = stock_pf + strategy_price[code] * pf_stock_num[code]

    cash_amount = initial_money - stock_amount

    backtest_df = pd.DataFrame({'주식포트폴리오':stock_pf})
    backtest_df['현금포트폴리오'] = [cash_amount] * len(backtest_df)
    backtest_df['종합포트폴리오'] = backtest_df['주식포트폴리오'] + backtest_df['현금포트폴리오']
    backtest_df['일변화율'] = backtest_df['종합포트폴리오'].pct_change()
    backtest_df['총변화율'] = backtest_df['종합포트폴리오']/initial_money - 1
    
    return backtest_df

#  [코드 5.16] 해당 날짜에 가격이 없으면 투자 관련 데이터에서 해당 종목 없애는 함수 (Ch5. 백테스트.ipynb)

def select_code_by_price(price_df, data_df, start_date):
    new_code_list = []
    for code in price_df[start_date].iloc[0].dropna().index:
        new_code_list.append(code)
        
    selected_df =  data_df.loc[new_code_list]
    return selected_df

# [코드 5.24] 백테스트 시작날짜가 주어지면 전략 기준 날짜를 계산하는 함수 (Ch5. 백테스트.ipynb)

def get_strategy_date(start_date):
    temp_year = int(start_date.split('-')[0])
    temp_month = start_date.split('-')[1]
    if temp_month in '1 2 3 4 5'.split(' '):
        strategy_date = str(temp_year - 2) + '/12'
    else:
        strategy_date = str(temp_year - 1) + '/12'
    return strategy_date


# [코드 5.32] 리밸런싱 백테스트 함수화 (Ch5. 백테스트.ipynb)

def backtest_re(strategy, start_date, end_date, initial_money, price_df, fr_df, fs_df, num, value_type=None, value_list=None, date_range=None):
    
    start_year = int(start_date.split('-')[0])
    end_year = int(end_date.split('-')[0])

    total_df = 0
    for temp in range(start_year, end_year):
        this_term_start = str(temp) + '-' + start_date.split('-')[1]
        this_term_end = str(temp+1) + '-' + start_date.split('-')[1]
        strategy_date = get_strategy_date(this_term_start)
        
        if strategy.__name__ == 'high_roa':
            st_df = strategy(select_code_by_price(price_df, fr_df, this_term_start), strategy_date, num)
        elif strategy.__name__ == 'magic_formula':
            st_df = strategy(select_code_by_price(price_df, invest_df, this_term_start), strategy_date, num)
        elif strategy.__name__ == 'get_value_rank':
            st_df = strategy(select_code_by_price(price_df, invest_df, this_term_start), value_type, strategy_date, num)
        elif strategy.__name__ == 'make_value_combo':
            st_df = strategy(value_list, select_code_by_price(price_df, invest_df, this_term_start), strategy_date, num)
        elif strategy.__name__ == 'get_fscore':
            st_df = strategy(select_code_by_price(price_df, fs_df, this_term_start), strategy_date, num)
        elif strategy.__name__ == 'get_momentum_rank':
            st_df = strategy(price_df, price_df[this_term_start].index[0] , date_range, num)
        elif strategy.__name__ == 'get_value_quality':
            st_df = strategy(select_code_by_price(price_df, invest_df, this_term_start), 
                             select_code_by_price(price_df, fs_df, this_term_start), strategy_date, num)
        
        backtest = backtest_beta(price_df, st_df, this_term_start, this_term_end, initial_money)
        temp_end = backtest[this_term_end].index[0]
        backtest = backtest[:temp_end]
        initial_money =  backtest['종합포트폴리오'][-1]
        if temp == start_year:
            total_df = backtest
        else:
            total_df = pd.concat([total_df[:-1], backtest])

    total_df ['일변화율'] = total_df ['종합포트폴리오'].pct_change()
    total_df ['총변화율'] = total_df ['종합포트폴리오']/ total_df ['종합포트폴리오'][0] - 1
    
    return total_df


# [코드 5.40] MDD 함수화 (Ch5. 백테스트.ipynb)

def get_mdd(back_test_df):
    max_list = [0]
    mdd_list = [0]

    for i in back_test_df.index[1:]:
        max_list.append(back_test_df['총변화율'][:i].max())
        if max_list[-1] > max_list[-2]:
            mdd_list.append(0)
        else:
            mdd_list.append(min(back_test_df['총변화율'][i] - max_list[-1], mdd_list[-1])   )

    back_test_df['max'] = max_list
    back_test_df['MDD'] = mdd_list
    
    return back_test_df































In [3]:
companies, prices, fs_df, fr_df, iv_df = load_all_data()

In [4]:
prices

,A095570,A068400,A006840,A027410,A282330,A138930,A001460,A001040,A079160,A000120,...,A200670,A065510,A243070,A084110,A145020,A024060,A010240,A189980,A037440,A238490
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2013-08-28,NaN,10850,38721,NaN,NaN,13281,194000,98540,46800,96500,...,NaN,18050,NaN,18360,NaN,2035,2025,NaN,5950,NaN
2013-08-29,NaN,11550,39205,NaN,NaN,13235,199500,98068,47600,99000,...,NaN,17850,NaN,18360,NaN,2340,2070,NaN,5980,NaN
2013-08-30,NaN,11900,40175,NaN,NaN,13281,190000,100426,48500,101000,...,NaN,18500,NaN,18647,NaN,2400,2060,NaN,5970,NaN
2013-09-02,NaN,11950,39932,NaN,NaN,13328,191000,99483,49600,99100,...,NaN,19600,NaN,19042,NaN,2400,2055,NaN,5970,NaN
2013-09-03,NaN,11700,39690,NaN,NaN,13934,190500,102312,49200,100000,...,NaN,19200,NaN,19042,NaN,2270,2138,NaN,5960,NaN
2013-09-04,NaN,11450,39157,NaN,NaN,13934,191000,101840,49400,99600,...,NaN,19350,NaN,18612,NaN,2610,2138,NaN,5940,NaN
2013-09-05,NaN,11500,38430,NaN,NaN,13934,191500,104669,52000,100000,...,NaN,18900,NaN,18826,NaN,3000,2130,NaN,5940,NaN
2013-09-06,NaN,11300,38769,NaN,NaN,13934,190500,104669,51700,99600,...,NaN,18450,NaN,18970,NaN,3095,2098,NaN,5930,NaN
2013-09-09,NaN,10950,38818,NaN,NaN,14074,190000,105612,52700,99600,...,NaN,18050,NaN,18934,NaN,3045,2118,NaN,5930,NaN


In [5]:
fs_df

2016/12                                                      \
             당기순이익     매출액      부채   영업이익 영업활동으로인한현금흐름      자본      자산   
Unnamed: 0                                                               
A095570     171     12539   16538   619    -1867        4275    20813    
A068400     68      6476    9233    327    -720         2344    11576    
A006840     1333    29218   17305   2137   3183         10114   27419    
A027410     1846    861     9754    80     3320         9422    19176    
A282330     NaN     NaN     NaN     NaN    NaN          NaN     NaN      
A138930     5181    NaN     863950  7123   -11258       70872   934822   
A001460     175     2119    3392    162    439          3939    7330     
A001040     5698    239542  157551  12529  20917        112544  270096   
A079160     56      14322   16213   703    1878         9205    25418    
A000120     682     60819   27819   2284   2084         27393   55212    
A011150     54      1737    645     59     -55          553     1197     
A097950     3535    145633  97058   8436   8001         60605   157662   
A037560     215     11006   8735    429    2758         9667    18402    
A000590     171     1231    320     198    166          2619    2939     
A012030     -207    2048    634     109    144          1515    2150     
A016610     64      NaN     56396   98     3077         6960    63356    
A005830     5338    NaN     405187  7261   31026        43447   448634   
A000990     883     7731    6449    1724   2483         3419    9868     
A139130     3019    NaN     494901  3869   326          39723   534624   
A004840     489     5513    2708    675    623          4304    7012     
A155660     107     2011    622     134    105          1335    1957     
A069730     87      1725    638     77     139          1264    1902     
A017940     -379    39959   23610   111    282          11832   35442    
A007700     302     4390    789     456    558          2267    3056     
A114090     1143    5482    2115    1512   1378         5187    7302     
A078930     9233    134624  114209  17542  8517         89203   203412   
A006360     -204    110356  100234  1430   812          33532   133766   
A001250     161     25538   9722    364    734          3543    13264    
A007070     2735    74020   23638   2181   4345         23575   47214    
A012630     3310    47499   30297   5172   8398         27549   57846    
...         ...       ...    ...    ...    ...           ...      ...    
A039010     105     878     260     107    64           458     718      
A204990     68      286     58      79     -1           448     505      
A092300     28      1191    604     51     182          605     1209     
A053660     17      1729    1926    88     152          111     2038     
A011080     -41     1276    550     4      -30          525     1075     
A060560     158     957     1678    1      40           2195    3873     
A064240     121     1221    318     177    99           1140    1458     
A039610     NaN     NaN     NaN     NaN    NaN          NaN     NaN      
A126640     38      2050    352     45     79           823     1175     
A086250     -139    292     490     2      -52          352     842      
A204630     -6      291     140     -6     -8           432     572      
A061250     64      1115    288     25     96           999     1286     
A134780     100     1133    463     78     136          881     1344     
A097870     NaN     NaN     NaN     NaN    NaN          NaN     NaN      
A050090     -34     430     229     12     -4           291     519      
A290270     16      119     29      17     18           36      65       
A090710     11      451     292     16     -18          135     426      
A205470     3       89      83      -1     6            125     209      
A115160     32      13505   5480    201    527          4703    10184    
A028080     -24     56      288     35     -3           1927    2215     
A200670     105

In [6]:
def make_fs_dataframe(firm_code):
    fs_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode=' + firm_code
    fs_page = requests.get(fs_url)
    fs_tables = pd.read_html(fs_page.text)
    
    temp_df = fs_tables[0]
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df[temp_df.columns[:4]]
    temp_df = temp_df.loc[['매출액', '영업이익', '당기순이익']]

    temp_df2 = fs_tables[2]
    temp_df2 = temp_df2.set_index(temp_df2.columns[0])
    temp_df2 = temp_df2.loc[['자산', '부채', '자본']]

    temp_df3 = fs_tables[4]
    temp_df3 = temp_df3.set_index(temp_df3.columns[0])
    temp_df3 = temp_df3.loc[['영업활동으로인한현금흐름']]

    fs_df = pd.concat([temp_df, temp_df2, temp_df3])
    
    return fs_df

In [48]:
fs_df = make_fs_dataframe('A095570')
fs_df

,2016/12,2017/12,2018/12,2019/06
IFRS(연결),,,,
매출액,12539.0,8439.0,10567.0,5744.0
영업이익,619.0,224.0,-213.0,179.0
당기순이익,171.0,206.0,373.0,977.0
자산,20813.0,23542.0,25751.0,17408.0
부채,16538.0,19071.0,20833.0,13424.0
자본,4275.0,4472.0,4918.0,3985.0
영업활동으로인한현금흐름,-1867.0,-1485.0,-748.0,-382.0


In [49]:
total_fs = change_df('A095570', fs_df)

In [97]:
fs_df2 = make_fs_dataframe('A006840')
total_fs2 = change_df('A006840', fs_df2)

In [98]:
total_fs2

2016/12                                                          2017/12                                                          2018/12                                                          2019/06                                                       
IFRS(연결)      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익  당기순이익       자산       부채       자본 영업활동으로인한현금흐름
A006840   29218.0  2137.0  1333.0  27419.0  17305.0  10114.0  3183.0       33925.0  2656.0  1854.0  29448.0  17927.0  11521.0  2809.0       37112.0  2697.0  2066.0  33979.0  19240.0  14739.0  3173.0       18876.0  1075.0  727.0  42695.0  27619.0  15076.0  1987.0

In [99]:
total_fs_merge = pd.concat([total_fs, total_fs2])

In [108]:
total_fs_merge

2016/12                                                          2017/12                                                          2018/12                                                          2019/06                                                       
IFRS(연결)      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익  당기순이익       자산       부채       자본 영업활동으로인한현금흐름
A095570   12539.0  619.0   171.0   20813.0  16538.0  4275.0  -1867.0       8439.0   224.0   206.0   23542.0  19071.0  4472.0  -1485.0       10567.0 -213.0   373.0   25751.0  20833.0  4918.0  -748.0        5744.0   179.0   977.0  17408.0  13424.0  3985.0  -382.0      
A006840   29218.0  2137.0  1333.0  27419.0  17305.0  10114.0  3183.0       33925.0  2656.0  1854.0  29448.0  17927.0  11521.0  2809.0       37112.0  2697.0  2066.0  33979.0  19240.0  14739.0  3173.0       18876.0  1075.0  727.0  42695.0  27619.0  15076.0  1987.0

In [109]:
fs_drop = total_fs_merge.drop('2019/06', axis=1)

In [133]:
fs_drop

2016/12                                                          2017/12                                                          2018/12                                                        
IFRS(연결)      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름
A095570   12539.0  619.0   171.0   20813.0  16538.0  4275.0  -1867.0       8439.0   224.0   206.0   23542.0  19071.0  4472.0  -1485.0       10567.0 -213.0   373.0   25751.0  20833.0  4918.0  -748.0      
A006840   29218.0  2137.0  1333.0  27419.0  17305.0  10114.0  3183.0       33925.0  2656.0  1854.0  29448.0  17927.0  11521.0  2809.0       37112.0  2697.0  2066.0  33979.0  19240.0  14739.0  3173.0

In [135]:
fs_df_origin = pd.DataFrame({'A095570':fs_drop.loc['A095570']}).T

In [118]:
fs_df

2016/12                                                      2017/12                                                       2018/12                                                      2019/06                                                     
IFRS(연결)      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름
A095570   12539.0  619.0  171.0  20813.0  16538.0  4275.0 -1867.0       8439.0  224.0  206.0  23542.0  19071.0  4472.0 -1485.0       10567.0 -213.0  373.0  25751.0  20833.0  4918.0 -748.0        5744.0  179.0  977.0  17408.0  13424.0  3985.0 -382.0

In [136]:
new_fs_merge = pd.concat([fs_df_origin, fs_df], axis=1)

In [140]:
new_fs_merge

2016/12                                                      2017/12                                                       2018/12                                                       2016/12                                                      2017/12                                                       2018/12                                                      2019/06                                                     
IFRS(연결)      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름
A095570   12539.0  619.0  171.0  20813.0  16538.0  4275.0 -1867.0       8439.0  224.0  206.0  23542.0  19071.0  4472.0 -1485.0       10567.0 -213.0  373.0  25751.0  20833.0  4918.0 -748.0        12539.0  619.0  171.0  20813.0  16538.0  4275.0 -1867.0       8439.0  224.0  206.0  23542.0  19071.0  4472.0 -1485.0       10567.0 -213.0  373.0  25751.0  20833.0  4918.0 -748.0        5744.0  179.0  977.0  17408.0  13424.0  3985.0 -382.0

In [138]:
new_fs_merge[new_fs_merge.columns[~new_fs_merge.columns.get_level_values(0).duplicated(keep='first')]]


2016/12          2017/12          2018/12          2019/06
IFRS(연결)      매출액      매출액     매출액     매출액      매출액      매출액     매출액
A095570   12539.0  12539.0  8439.0  8439.0  10567.0  10567.0  5744.0

In [141]:
fs_df_added = new_fs_merge[new_fs_merge.columns[~new_fs_merge.columns.duplicated(keep=False)]]


In [142]:
final_fs_merge = pd.concat([fs_df_origin, fs_df_added], axis=1)

In [143]:
final_fs_merge

2016/12                                                      2017/12                                                       2018/12                                                      2019/06                                                     
IFRS(연결)      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름
A095570   12539.0  619.0  171.0  20813.0  16538.0  4275.0 -1867.0       8439.0  224.0  206.0  23542.0  19071.0  4472.0 -1485.0       10567.0 -213.0  373.0  25751.0  20833.0  4918.0 -748.0        5744.0  179.0  977.0  17408.0  13424.0  3985.0 -382.0

In [144]:
fs_drop

2016/12                                                          2017/12                                                          2018/12                                                        
IFRS(연결)      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름
A095570   12539.0  619.0   171.0   20813.0  16538.0  4275.0  -1867.0       8439.0   224.0   206.0   23542.0  19071.0  4472.0  -1485.0       10567.0 -213.0   373.0   25751.0  20833.0  4918.0  -748.0      
A006840   29218.0  2137.0  1333.0  27419.0  17305.0  10114.0  3183.0       33925.0  2656.0  1854.0  29448.0  17927.0  11521.0  2809.0       37112.0  2697.0  2066.0  33979.0  19240.0  14739.0  3173.0

In [145]:
fs_drop_temp = fs_drop.drop('A095570')

In [146]:
fs_drop_temp

2016/12                                                          2017/12                                                          2018/12                                                        
IFRS(연결)      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름
A006840   29218.0  2137.0  1333.0  27419.0  17305.0  10114.0  3183.0       33925.0  2656.0  1854.0  29448.0  17927.0  11521.0  2809.0       37112.0  2697.0  2066.0  33979.0  19240.0  14739.0  3173.0

In [147]:
fs_final = pd.concat([fs_drop_temp, final_fs_merge])

In [148]:
fs_final

2016/12                                                          2017/12                                                          2018/12                                                          2019/06                                                      
IFRS(연결)   당기순이익      매출액       부채    영업이익 영업활동으로인한현금흐름       자본       자산   당기순이익      매출액       부채    영업이익 영업활동으로인한현금흐름       자본       자산   당기순이익      매출액       부채    영업이익 영업활동으로인한현금흐름       자본       자산   당기순이익     매출액       부채   영업이익 영업활동으로인한현금흐름      자본       자산
A006840   1333.0  29218.0  17305.0  2137.0  3183.0       10114.0  27419.0  1854.0  33925.0  17927.0  2656.0  2809.0       11521.0  29448.0  2066.0  37112.0  19240.0  2697.0  3173.0       14739.0  33979.0 NaN     NaN     NaN      NaN    NaN          NaN     NaN     
A095570   171.0   12539.0  16538.0  619.0  -1867.0       4275.0   20813.0  206.0   8439.0   19071.0  224.0  -1485.0       4472.0   23542.0  373.0   10567.0  20833.0 -213.0  -748.0        4918.0   25751.0  977.0   5744.0  13424.0  179.0 -382.0        3985.0  17408.0

In [151]:
fs_df_origin = pd.DataFrame({'A006840':fs_final.loc['A006840']}).T

In [152]:
fs_df_origin

2016/12                                                          2017/12                                                          2018/12                                                          2019/06                                  
IFRS(연결)   당기순이익      매출액       부채    영업이익 영업활동으로인한현금흐름       자본       자산   당기순이익      매출액       부채    영업이익 영업활동으로인한현금흐름       자본       자산   당기순이익      매출액       부채    영업이익 영업활동으로인한현금흐름       자본       자산   당기순이익 매출액  부채 영업이익 영업활동으로인한현금흐름  자본  자산
A006840   1333.0  29218.0  17305.0  2137.0  3183.0       10114.0  27419.0  1854.0  33925.0  17927.0  2656.0  2809.0       11521.0  29448.0  2066.0  37112.0  19240.0  2697.0  3173.0       14739.0  33979.0 NaN     NaN NaN NaN  NaN          NaN NaN

In [154]:
total_fs2

2016/12                                                          2017/12                                                          2018/12                                                          2019/06                                                       
IFRS(연결)      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익  당기순이익       자산       부채       자본 영업활동으로인한현금흐름
A006840   29218.0  2137.0  1333.0  27419.0  17305.0  10114.0  3183.0       33925.0  2656.0  1854.0  29448.0  17927.0  11521.0  2809.0       37112.0  2697.0  2066.0  33979.0  19240.0  14739.0  3173.0       18876.0  1075.0  727.0  42695.0  27619.0  15076.0  1987.0

In [155]:
new_fs_merge = pd.concat([fs_df_origin, total_fs2], axis=1)

In [160]:
new_fs_merge = pd.concat([fs_df_origin, total_fs2], axis=1)


In [161]:
new_fs_merge

2016/12                                                          2017/12                                                          2018/12                                                          2019/06                                    2016/12                                                          2017/12                                                          2018/12                                                          2019/06                                                       
IFRS(연결)   당기순이익      매출액       부채    영업이익 영업활동으로인한현금흐름       자본       자산   당기순이익      매출액       부채    영업이익 영업활동으로인한현금흐름       자본       자산   당기순이익      매출액       부채    영업이익 영업활동으로인한현금흐름       자본       자산   당기순이익 매출액  부채 영업이익 영업활동으로인한현금흐름  자본  자산      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익  당기순이익       자산       부채       자본 영업활동으로인한현금흐름
A006840   1333.0  29218.0  17305.0  2137.0  3183.0       10114.0  27419.0  1854.0  33925.0  17927.0  2656.0  2809.0       11521.0  29448.0  2066.0  37112.0  19240.0  2697.0  3173.0       14739.0  33979.0 NaN     NaN NaN NaN  NaN          NaN NaN  29218.0  2137.0  1333.0  27419.0  17305.0  10114.0  3183.0       33925.0  2656.0  1854.0  29448.0  17927.0  11521.0  2809.0       37112.0  2697.0  2066.0  33979.0  19240.0  14739.0  3173.0       18876.0  1075.0  727.0  42695.0  27619.0  15076.0  1987.0

In [168]:
new_fs_merge.drop('2016/12', axis=1, level=0)

2017/12                                                          2018/12                                                          2019/06                                    2017/12                                                          2018/12                                                          2019/06                                                       
IFRS(연결)   당기순이익      매출액       부채    영업이익 영업활동으로인한현금흐름       자본       자산   당기순이익      매출액       부채    영업이익 영업활동으로인한현금흐름       자본       자산   당기순이익 매출액  부채 영업이익 영업활동으로인한현금흐름  자본  자산      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익  당기순이익       자산       부채       자본 영업활동으로인한현금흐름
A006840   1854.0  33925.0  17927.0  2656.0  2809.0       11521.0  29448.0  2066.0  37112.0  19240.0  2697.0  3173.0       14739.0  33979.0 NaN     NaN NaN NaN  NaN          NaN NaN  33925.0  2656.0  1854.0  29448.0  17927.0  11521.0  2809.0       37112.0  2697.0  2066.0  33979.0  19240.0  14739.0  3173.0       18876.0  1075.0  727.0  42695.0  27619.0  15076.0  1987.0

In [175]:
new_fs_merge

2016/12                                                          2017/12                                                          2018/12                                                          2019/06                                    2016/12                                                          2017/12                                                          2018/12                                                          2019/06                                                       
IFRS(연결)   당기순이익      매출액       부채    영업이익 영업활동으로인한현금흐름       자본       자산   당기순이익      매출액       부채    영업이익 영업활동으로인한현금흐름       자본       자산   당기순이익      매출액       부채    영업이익 영업활동으로인한현금흐름       자본       자산   당기순이익 매출액  부채 영업이익 영업활동으로인한현금흐름  자본  자산      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익  당기순이익       자산       부채       자본 영업활동으로인한현금흐름
A006840   1333.0  29218.0  17305.0  2137.0  3183.0       10114.0  27419.0  1854.0  33925.0  17927.0  2656.0  2809.0       11521.0  29448.0  2066.0  37112.0  19240.0  2697.0  3173.0       14739.0  33979.0 NaN     NaN NaN NaN  NaN          NaN NaN  29218.0  2137.0  1333.0  27419.0  17305.0  10114.0  3183.0       33925.0  2656.0  1854.0  29448.0  17927.0  11521.0  2809.0       37112.0  2697.0  2066.0  33979.0  19240.0  14739.0  3173.0       18876.0  1075.0  727.0  42695.0  27619.0  15076.0  1987.0

In [174]:
new_fs_merge.columns.get_level_values(0).duplicated()

array([False,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [191]:
~new_fs_merge.columns.duplicated(keep='last')

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [192]:
pd.Series(~new_fs_merge.columns.duplicated(keep='last'))

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    True 
29    True 
30    True 
31    True 
32    True 
33    True 
34    True 
35    True 
36    True 
37    True 
38    True 
39    True 
40    True 
41    True 
42    True 
43    True 
44    True 
45    True 
46    True 
47    True 
48    True 
49    True 
50    True 
51    True 
52    True 
53    True 
54    True 
55    True 
dtype: bool

In [194]:
new_fs_merge.columns[pd.Series(~new_fs_merge.columns.duplicated(keep='last'))]

MultiIndex(levels=[['2016/12', '2017/12', '2018/12', '2019/06'], ['당기순이익', '매출액', '부채', '영업이익', '영업활동으로인한현금흐름', '자본', '자산']],
           codes=[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3], [1, 3, 0, 6, 2, 5, 4, 1, 3, 0, 6, 2, 5, 4, 1, 3, 0, 6, 2, 5, 4, 1, 3, 0, 6, 2, 5, 4]],
           names=[None, 'IFRS(연결)'])

In [195]:
new_fs_merge.loc[:,~new_fs_merge.columns.duplicated(keep='last')]

2016/12                                                          2017/12                                                          2018/12                                                          2019/06                                                       
IFRS(연결)      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익  당기순이익       자산       부채       자본 영업활동으로인한현금흐름
A006840   29218.0  2137.0  1333.0  27419.0  17305.0  10114.0  3183.0       33925.0  2656.0  1854.0  29448.0  17927.0  11521.0  2809.0       37112.0  2697.0  2066.0  33979.0  19240.0  14739.0  3173.0       18876.0  1075.0  727.0  42695.0  27619.0  15076.0  1987.0

In [182]:
new_fs_merge[new_fs_merge.columns[~new_fs_merge.columns.duplicated(keep='last')]]

2016/12                                                                                                                           2017/12                                                                                                                           2018/12                                                                                                                          2019/06                                                                                                    
IFRS(연결)      매출액      매출액    영업이익    영업이익   당기순이익   당기순이익       자산       자산       부채       부채       자본       자본 영업활동으로인한현금흐름 영업활동으로인한현금흐름      매출액      매출액    영업이익    영업이익   당기순이익   당기순이익       자산       자산       부채       부채       자본       자본 영업활동으로인한현금흐름 영업활동으로인한현금흐름      매출액      매출액    영업이익    영업이익   당기순이익   당기순이익       자산       자산       부채       부채       자본       자본 영업활동으로인한현금흐름 영업활동으로인한현금흐름     매출액      매출액 영업이익    영업이익 당기순이익  당기순이익  자산       자산  부채       부채  자본       자본 영업활동으로인한현금흐름 영업활동으로인한현금흐름
A006840   29218.0  29218.0  2137.0  2137.0  1333.0  1333.0  27419.0  27419.0  17305.0  17305.0  10114.0  10114.0  3183.0       3183.0       33925.0  33925.0  2656.0  2656.0  1854.0  1854.0  29448.0  29448.0  17927.0  17927.0  11521.0  11521.0  2809.0       2809.0       37112.0  37112.0  2697.0  2697.0  2066.0  2066.0  33979.0  33979.0  19240.0  19240.0  14739.0  14739.0  3173.0       3173.0      NaN      18876.0 NaN   1075.0 NaN    727.0 NaN  42695.0 NaN  27619.0 NaN  15076.0 NaN           1987.0

In [176]:
new_fs_merge[new_fs_merge.columns[~new_fs_merge.columns.duplicated(keep='last')]]

2016/12                                                                                                                           2017/12                                                                                                                           2018/12                                                                                                                          2019/06                                                                                                    
IFRS(연결)      매출액      매출액    영업이익    영업이익   당기순이익   당기순이익       자산       자산       부채       부채       자본       자본 영업활동으로인한현금흐름 영업활동으로인한현금흐름      매출액      매출액    영업이익    영업이익   당기순이익   당기순이익       자산       자산       부채       부채       자본       자본 영업활동으로인한현금흐름 영업활동으로인한현금흐름      매출액      매출액    영업이익    영업이익   당기순이익   당기순이익       자산       자산       부채       부채       자본       자본 영업활동으로인한현금흐름 영업활동으로인한현금흐름     매출액      매출액 영업이익    영업이익 당기순이익  당기순이익  자산       자산  부채       부채  자본       자본 영업활동으로인한현금흐름 영업활동으로인한현금흐름
A006840   29218.0  29218.0  2137.0  2137.0  1333.0  1333.0  27419.0  27419.0  17305.0  17305.0  10114.0  10114.0  3183.0       3183.0       33925.0  33925.0  2656.0  2656.0  1854.0  1854.0  29448.0  29448.0  17927.0  17927.0  11521.0  11521.0  2809.0       2809.0       37112.0  37112.0  2697.0  2697.0  2066.0  2066.0  33979.0  33979.0  19240.0  19240.0  14739.0  14739.0  3173.0       3173.0      NaN      18876.0 NaN   1075.0 NaN    727.0 NaN  42695.0 NaN  27619.0 NaN  15076.0 NaN           1987.0

In [164]:
# fs_df_added = new_fs_merge[new_fs_merge.columns[~new_fs_merge.columns.duplicated(keep=False)]]
new_fs_merge[new_fs_merge.columns[~new_fs_merge.columns.duplicated(keep='last')]]

2016/12                                                                                                                           2017/12                                                                                                                           2018/12                                                                                                                          2019/06                                                                                                    
IFRS(연결)      매출액      매출액    영업이익    영업이익   당기순이익   당기순이익       자산       자산       부채       부채       자본       자본 영업활동으로인한현금흐름 영업활동으로인한현금흐름      매출액      매출액    영업이익    영업이익   당기순이익   당기순이익       자산       자산       부채       부채       자본       자본 영업활동으로인한현금흐름 영업활동으로인한현금흐름      매출액      매출액    영업이익    영업이익   당기순이익   당기순이익       자산       자산       부채       부채       자본       자본 영업활동으로인한현금흐름 영업활동으로인한현금흐름     매출액      매출액 영업이익    영업이익 당기순이익  당기순이익  자산       자산  부채       부채  자본       자본 영업활동으로인한현금흐름 영업활동으로인한현금흐름
A006840   29218.0  29218.0  2137.0  2137.0  1333.0  1333.0  27419.0  27419.0  17305.0  17305.0  10114.0  10114.0  3183.0       3183.0       33925.0  33925.0  2656.0  2656.0  1854.0  1854.0  29448.0  29448.0  17927.0  17927.0  11521.0  11521.0  2809.0       2809.0       37112.0  37112.0  2697.0  2697.0  2066.0  2066.0  33979.0  33979.0  19240.0  19240.0  14739.0  14739.0  3173.0       3173.0      NaN      18876.0 NaN   1075.0 NaN    727.0 NaN  42695.0 NaN  27619.0 NaN  15076.0 NaN           1987.0

In [127]:
new_fs_merge.drop_duplicates(subset=list(new_fs_merge.columns.get_level_values(0)))

KeyError: Index(['2016/12', '2017/12', '2018/12', '2019/06'], dtype='object')

In [ ]:
new_fs_merge_duplicate = new_fs_merge.loc[~new_fs_merge.columns.duplicated(keep='last')]

In [67]:
fs_year_frag = fs_df['2018/12']

In [75]:
fs_year_frag.columns.values

array(['매출액', '영업이익', '당기순이익', '자산', '부채', '자본', '영업활동으로인한현금흐름'],
      dtype=object)

In [78]:
# temp_df.columns = [[col]*len(dataframe), temp_df.columns]
fs_year_frag.columns = [['2018/12']*len(fs_year_frag.columns), fs_year_frag.columns]

In [79]:
fs_year_frag

2018/12                                                     
IFRS(연결)      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름
A095570   10567.0 -213.0  373.0  25751.0  20833.0  4918.0 -748.0

In [66]:
fs_df.columns[0]

('2016/12', '매출액')

In [80]:
fs_drop

2016/12                                                      2017/12                                                       2018/12                                                     
IFRS(연결)      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름
A095570   12539.0  619.0  171.0  20813.0  16538.0  4275.0 -1867.0       8439.0  224.0  206.0  23542.0  19071.0  4472.0 -1485.0       10567.0 -213.0  373.0  25751.0  20833.0  4918.0 -748.0

In [81]:
fs_df

2016/12                                                      2017/12                                                       2018/12                                                      2019/06                                                     
IFRS(연결)      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름
A095570   12539.0  619.0  171.0  20813.0  16538.0  4275.0 -1867.0       8439.0  224.0  206.0  23542.0  19071.0  4472.0 -1485.0       10567.0 -213.0  373.0  25751.0  20833.0  4918.0 -748.0        5744.0  179.0  977.0  17408.0  13424.0  3985.0 -382.0

In [103]:
fs_merge

2016/12                                                      2017/12                                                       2018/12                                                      2019/06                                                     
IFRS(연결)      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름
A095570   12539.0  619.0  171.0  20813.0  16538.0  4275.0 -1867.0       8439.0  224.0  206.0  23542.0  19071.0  4472.0 -1485.0       10567.0 -213.0  373.0  25751.0  20833.0  4918.0 -748.0        5744.0  179.0  977.0  17408.0  13424.0  3985.0 -382.0

In [104]:
fs_merge = fs_drop

In [105]:
years = fs_df.columns.get_level_values(0).unique()
for year in years:
    if year not in fs_merge:
        print('merge' + year)
        fs_year_frag = fs_df[year]
        fs_year_frag.columns = [[year]*len(fs_year_frag.columns), fs_year_frag.columns]
#         fs_merge = pd.concat([fs_merge, fs_year_frag], axis=1)
        fs_merge = pd.merge(fs_merge, fs_year_frag, left_index=True, right_index=True, how='outer')


merge2019/06


In [106]:
fs_merge

2016/12                                                          2017/12                                                          2018/12                                                         2019/06                                                     
IFRS(연결)      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름      매출액    영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름
A006840   29218.0  2137.0  1333.0  27419.0  17305.0  10114.0  3183.0       33925.0  2656.0  1854.0  29448.0  17927.0  11521.0  2809.0       37112.0  2697.0  2066.0  33979.0  19240.0  14739.0  3173.0      NaN     NaN    NaN    NaN      NaN      NaN     NaN         
A095570   12539.0  619.0   171.0   20813.0  16538.0  4275.0  -1867.0       8439.0   224.0   206.0   23542.0  19071.0  4472.0  -1485.0       10567.0 -213.0   373.0   25751.0  20833.0  4918.0  -748.0        5744.0  179.0  977.0  17408.0  13424.0  3985.0 -382.0

In [107]:
years = fs_df2.columns.get_level_values(0).unique()
for year in years:
    if year not in fs_merge:
        print('merge' + year)
        fs_year_frag = fs_df2[year]
        fs_year_frag.columns = [[year]*len(fs_year_frag.columns), fs_year_frag.columns]
#         fs_merge = pd.concat([fs_merge, fs_year_frag], axis=1)
        fs_merge = pd.merge(fs_merge, fs_year_frag, left_index=True, right_index=True, how='outer')


In [61]:
fs_merge

2016/12                                                      2017/12                                                       2018/12                                                      2019/06                                                       2016/12                                                      2017/12                                                       2018/12                                                      2019/06                                                     
IFRS(연결)      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름
A095570   12539.0  619.0  171.0  20813.0  16538.0  4275.0 -1867.0       8439.0  224.0  206.0  23542.0  19071.0  4472.0 -1485.0       10567.0 -213.0  373.0  25751.0  20833.0  4918.0 -748.0        5744.0  179.0  977.0  17408.0  13424.0  3985.0 -382.0        12539.0  619.0  171.0  20813.0  16538.0  4275.0 -1867.0       8439.0  224.0  206.0  23542.0  19071.0  4472.0 -1485.0       10567.0 -213.0  373.0  25751.0  20833.0  4918.0 -748.0        5744.0  179.0  977.0  17408.0  13424.0  3985.0 -382.0

In [46]:
year = fs_df.columns[0][0]
if year not in total_fs:
    fs_merge = pd.concat([total_fs, fs_df], axis=1)

In [47]:
fs_merge

2016/12                                                      2017/12                                                       2018/12                                                      2019/06                                                       2016/12                                                      2017/12                                                       2018/12                                                      2019/06                                                     
IFRS(연결)      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름
A095570   12539.0  619.0  171.0  20813.0  16538.0  4275.0 -1867.0       8439.0  224.0  206.0  23542.0  19071.0  4472.0 -1485.0       10567.0 -213.0  373.0  25751.0  20833.0  4918.0 -748.0        5744.0  179.0  977.0  17408.0  13424.0  3985.0 -382.0        12539.0  619.0  171.0  20813.0  16538.0  4275.0 -1867.0       8439.0  224.0  206.0  23542.0  19071.0  4472.0 -1485.0       10567.0 -213.0  373.0  25751.0  20833.0  4918.0 -748.0        5744.0  179.0  977.0  17408.0  13424.0  3985.0 -382.0

In [31]:
# pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [45]:
year = fs_df.columns[0][0]
year = '2016/12'
year in fs_df

True

In [41]:
fs_df['2016/12']

IFRS(연결),매출액,영업이익,당기순이익,자산,부채,자본,영업활동으로인한현금흐름
A095570,12539.0,619.0,171.0,20813.0,16538.0,4275.0,-1867.0


In [32]:
fs_merge


2016/12                                                      2017/12                                                       2018/12                                                      2019/06                                                       2016/12                                                      2017/12                                                       2018/12                                                      2019/06                                                     
IFRS(연결)      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름      매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름     매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름
A095570   12539.0  619.0  171.0  20813.0  16538.0  4275.0 -1867.0       8439.0  224.0  206.0  23542.0  19071.0  4472.0 -1485.0       10567.0 -213.0  373.0  25751.0  20833.0  4918.0 -748.0        5744.0  179.0  977.0  17408.0  13424.0  3985.0 -382.0        12539.0  619.0  171.0  20813.0  16538.0  4275.0 -1867.0       8439.0  224.0  206.0  23542.0  19071.0  4472.0 -1485.0       10567.0 -213.0  373.0  25751.0  20833.0  4918.0 -748.0        5744.0  179.0  977.0  17408.0  13424.0  3985.0 -382.0

In [25]:
fs_merge = pd.merge(total_fs, fs_df, left_index=True, right_index=True, how='outer')

In [26]:
fs_merge

2016/12_x                                                       \
IFRS(연결)       매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름   
A095570   12539.0   619.0  171.0  20813.0  16538.0  4275.0 -1867.0        

         2017/12_x                ... 2018/12_y                       \
IFRS(연결)       매출액   영업이익  당기순이익  ...        부채      자본 영업활동으로인한현금흐름   
A095570   8439.0    224.0  206.0  ...  20833.0   4918.0 -748.0         

         2019/06_y                                                       
IFRS(연결)       매출액   영업이익  당기순이익       자산       부채      자본 영업활동으로인한현금흐름  
A095570   5744.0    179.0  977.0  17408.0  13424.0  3985.0 -382.0        

[1 rows x 56 columns]

In [ ]:


for num, code in enumerate(companies['종목코드']):
    try:
        print(num, code)
        time.sleep(1)
        try:
            fs_df = make_fs_dataframe(code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            fs_df = make_fs_dataframe(code)
        fs_df_changed = change_df(code, fs_df)
        if num == 0 :
            total_fs = fs_df_changed
        else:
            total_fs = pd.concat([total_fs, fs_df_changed])
    except ValueError:
        continue
    except KeyError:
        continue

In [ ]:


        
def update_fs_data(company_df, origin_fs_df)
    fs_merge = origin_fs_df
    for num, code in enumerate(company_df.index):
        try:
            print(num, code)
            time.sleep(1)
            try:
                fs_df = make_fs_dataframe(code)
            except requests.exceptions.Timeout:
                time.sleep(60)
                fs_df = make_fs_dataframe(code)
            fs_df_changed = change_df(code, fs_df)
            
            years = fs_df_changed.columns.get_level_values(0).unique()
            for year in years:
                if year not in fs_merge:
                    print('merge' + year)
                    fs_year_frag = fs_df_changed[year]
                    fs_year_frag.columns = [[year]*len(fs_year_frag.columns), fs_year_frag.columns]
                    fs_merge = pd.concat([fs_merge, fs_year_frag], axis=1)
        except ValueError:
            continue
        except KeyError:
            continue
    return fs_merge

